In [1]:
import numpy as np
import torch
import torch.nn as nn
# import torch.nn.functional as F
import torch.utils.data as data
import torch.optim as optim
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import wandb

In [2]:
%load_ext autoreload
%autoreload 2
from models import VisualEncoder, VisualDecoder

In [3]:
images = np.load('data/images.npy') # путь до картинок из иглу
images = np.transpose(images,(0,3,1,2))
images = torch.Tensor(images)
train_dataset = images
train_set, val_set = torch.utils.data.random_split(train_dataset, [100000, 13873])
train_loader = data.DataLoader(train_set, batch_size=256, shuffle=True, drop_last=True, pin_memory=False, num_workers=4)
val_loader = data.DataLoader(val_set, batch_size=256, shuffle=False, drop_last=False, num_workers=4)

## Training AtariCNN

In [6]:
class Autoencoder(nn.Module):
    def __init__(self,
               encoder_class : object = VisualEncoder,
               decoder_class : object = VisualDecoder):
        super().__init__()
        self.encoder = encoder_class()
        self.decoder = decoder_class()
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return decode
model = Autoencoder()
model.to('cuda')

total_params = sum(p.numel() for p in model.encoder.parameters() if p.requires_grad)
print('Total params:', total_params)

Total params: 600736


In [ ]:
config_defaults = {
    'epochs': 50,
    'batch_size': 1024,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'scheduler_step_size': 30,
    'scheduler_gamma': 0.5
}

wandb.init(project='AutoEncoder', entity='neuro_ai', name='AtariCNN', config=config_defaults)
config = wandb.config
optimizer = optim.Adam(model.parameters(), config.learning_rate)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                            step_size=config.scheduler_step_size, 
                                            gamma=config.scheduler_gamma)

for epoch in range(config.epochs):
    #train
    model.train()
    train_loss = []
    for data in train_loader:
        data /= 255
        data = data.to('cuda')
        predict = model(data)
        loss = criterion(predict, data)    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    train_loss = np.array(train_loss).mean()
    
    #evaluate
    model.eval()
    val_loss = []
    for data in val_loader:
        data /= 255
        data = data.to('cuda')
        predict = model(data)
        loss = criterion(predict, data)
        val_loss.append(loss.item())
    val_loss = np.array(val_loss).mean()
    scheduler.step()
    wandb.log({"train_loss":train_loss, 'val_loss': val_loss})

In [ ]:
torch.save(model.encoder.state_dict(), 'models/AtariCNN/encoder_weigths.pth')
torch.save(model.decoder.state_dict(), 'models/AtariCNN/decoder_weigths.pth')

##  AnnaCNN 

In [13]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 512, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

class VisualDecoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.ConvTranspose2d(512, 64, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=0, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=8, stride=4, padding=0),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = x.reshape(x.shape[0], 512, 1, 1)
        x = self.cnn(x)
        return x 

In [14]:
class Autoencoder(nn.Module):
    def __init__(self,
               encoder_class : object = VisualEncoder,
               decoder_class : object = VisualDecoder):
        super().__init__()
        self.encoder = encoder_class()
        self.decoder = decoder_class()
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return decode
model = Autoencoder()
model.to('cuda')

total_params = sum(p.numel() for p in model.encoder.parameters() if p.requires_grad)
print('Total params:', total_params)

Total params: 244448


In [ ]:
config_defaults = {
    'epochs': 100,
    'batch_size': 1024,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'scheduler_step_size': 30,
    'scheduler_gamma': 0.5
}

wandb.init(project='AutoEncoder', entity='neuro_ai', name='AnnaCNN', config=config_defaults)
config = wandb.config
optimizer = optim.Adam(model.parameters(), config.learning_rate)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                            step_size=config.scheduler_step_size, 
                                            gamma=config.scheduler_gamma)

for epoch in range(config.epochs):
    #train
    model.train()
    train_loss = []
    for data in train_loader:
        data /= 255
        data = data.to('cuda')
        predict = model(data)
        loss = criterion(predict, data)    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    train_loss = np.array(train_loss).mean()
    
    #evaluate
    model.eval()
    val_loss = []
    for data in val_loader:
        data /= 255
        data = data.to('cuda')
        predict = model(data)
        loss = criterion(predict, data)
        val_loss.append(loss.item())
    val_loss = np.array(val_loss).mean()
    scheduler.step()
    wandb.log({"train_loss":train_loss, 'val_loss': val_loss})

train_loss,0.02454
val_loss,0.02426


train_loss,█▂▂▁▁▁▁▁
val_loss,█▄▃▃▂▂▁▁


In [16]:
torch.save(model.encoder.state_dict(), 'models/AnnaCNN/encoder_weigths.pth')
torch.save(model.decoder.state_dict(), 'models/AnnaCNN/decoder_weigths.pth')

##  AlinaCNN 

In [17]:
class VisualEncoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=8, stride=4, padding=0),  
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0), 
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(), 
            nn.Conv2d(64, 64, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.Flatten(),
        )

    def forward(self, x):
        return self.cnn(x)

class VisualDecoder(nn.Module):
    def __init__(self):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.ConvTranspose2d(64, 64, kernel_size=2, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, kernel_size=3, stride=1, padding=0),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=0, output_padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 3, kernel_size=8, stride=4, padding=0),
            nn.Tanh()
        )
    
    def forward(self, x):
        x = x.reshape(x.shape[0], 64, 1, 1)
        x = self.cnn(x)
        return x 

In [18]:
class Autoencoder(nn.Module):
    def __init__(self,
               encoder_class : object = VisualEncoder,
               decoder_class : object = VisualDecoder):
        super().__init__()
        self.encoder = encoder_class()
        self.decoder = decoder_class()
    def forward(self, x):
        encode = self.encoder(x)
        decode = self.decoder(encode)
        return decode
model = Autoencoder()
model.to('cuda')

total_params = sum(p.numel() for p in model.encoder.parameters() if p.requires_grad)
print('Total params:', total_params)

Total params: 129312


In [ ]:
config_defaults = {
    'epochs': 100,
    'batch_size': 1024,
    'learning_rate': 1e-3,
    'optimizer': 'adam',
    'scheduler_step_size': 30,
    'scheduler_gamma': 0.1
}

wandb.init(project='AutoEncoder', entity='neuro_ai', name='AlinaCNN', config=config_defaults)
config = wandb.config
optimizer = optim.Adam(model.parameters(), config.learning_rate)
criterion = torch.nn.MSELoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 
                                            step_size=config.scheduler_step_size, 
                                            gamma=config.scheduler_gamma)

for epoch in range(config.epochs):
    #train
    model.train()
    train_loss = []
    for data in train_loader:
        data /= 255
        data = data.to('cuda')
        predict = model(data)
        loss = criterion(predict, data)    
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss.append(loss.item())
    train_loss = np.array(train_loss).mean()
    
    #evaluate
    model.eval()
    val_loss = []
    for data in val_loader:
        data /= 255
        data = data.to('cuda')
        predict = model(data)
        loss = criterion(predict, data)
        val_loss.append(loss.item())
    val_loss = np.array(val_loss).mean()
    scheduler.step()
    wandb.log({"train_loss":train_loss, 'val_loss': val_loss})

train_loss,0.01751
val_loss,0.01761


train_loss,█▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▅▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


In [ ]:
torch.save(model.encoder.state_dict(), 'models/AlinaCNN/encoder_weigths.pth')
torch.save(model.decoder.state_dict(), 'models/AlinaCNN/decoder_weigths.pth')